This is probably the simplest tutorial for Titanic. It is narrated in a nice story like style. Hope you like it. First we will generate the o/p in 1 line of code (without ML). After that we will use a simple ML model to arrive at a decision. Then we will make it more complex. add couple of other models..create a simple ensemble etc. We will 'evolve' the code. 

Edit: 10 Sep: If you like this notebook, you may also like a similar one: https://www.kaggle.com/allohvk/111-couples-on-the-titanic-a-recipe-for-disaster which is a fun notebook less than 25 lines on how many couples were travelling abord teh Titanic and whether we find Rose and Jack amongst them!

In [ ]:
import pandas as pd
test_data = pd.read_csv ('/kaggle/input/titanic/test.csv')
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

We know that females are given preference. Our first iteration is very simple and we wont even use an ML. We will simply assume all males expire and all females live. We do it in 1 line.

In [ ]:
test_data['Survived'] = test_data.apply(lambda x: 0 if x.Sex=='male'else 1, axis=1)

On the left is the new column indicating Survival. On the right we have used the ***dataframe.apply*** function. It applies a 'certain logic' on all rows of the dataframe. What is this 'certain logic' it applies? Well it is left to us to define that logic. This 'certain logic is executed for each row of the dataframe which is represented by the variable x. In this case the logic is simple - *if x.Sex is male return expired else return survived*.

We dont have to loop or anything. Pandas does all that. With one line we are able to add a new column indicating whether passenger survived or not survived.

Now we convert the PassngerID and the new 'Survived' col back to a CSV file which we will upload to Kaggle as our first submission. We dont want the index or any other columns to be appearing as a part of the CSV file.

In [ ]:
test_data[['PassengerId','Survived']].to_csv('KaggleOutput', index=False)

['PassengerId','Survived'] is a Python list. This is fed to the external [ ]. Pandas will then automatically trim the dataframe to retain these 2 columns only. Kaggle does not expect all the other columns during submission. That is it. Export to CSV

Search for the 'KaggleOutput.csv' file and submit it to Kaggle. It takes less than 10 seconds. **Voila - You get a decent 76.6%**. The drumbeats roll. It is a sombre moment as your score is reflected in the leaderboard. The dashboard politely hides the last 2-3 digits of your ranking as if to encourage you. You can do better. Let us now take the help of a model. Decision tree is a good bet if not for anything but for the nice output it generates explaining its internal thought process. That is very convinient because we can 'peep' into the model to see how it arrived at its decision for each passenger in the test dataset. It is also like the 10 question game we used to play as kids...where one person thinks of a name or place and the other person has to guess what the first person is thinking with 10 Yes/No questions 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

##model=DecisionTreeClassifier().fit(train_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']],train_data['Survived'])

##model.predict(test_data['Survived'])
##And a big error is thrown if above line were uncommented

Let us not worry about the error. It will be easily fixed. But first let us look at what we wanted to do.

We 'fit' on the train_data and then predict on the test_data. Notice the ***fit (training dataframe, label dataframe)*** syntax. Here the training dataframe is train_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]. This is the original training data but we have not selected the PassengerId which is sort of an index and Name and Ticket and Embarked which we intuitively feel may not have any bearing on the survival. Cabin number too has lot of nulls and is dropped. The label dataframe contains only the 'Survived' column of the train_data. 

After the training (fitting) is done, we do the 'prediction' using the above trained model. 

Anyway the above statement throws an error..something about not being able to convert String to float. Well 'text' values cannot be used in a decision tree model or any model for that matter. Models work using linear algebra. Linear algebra needs numbers not alphabets. We need to find a way to convert this 'male' and 'female' into 0's and 1's. This is done in one line of code.  

In [ ]:
for df in [train_data,test_data]:
    df['Sex_boolean']=df['Sex'].map({'male':1,'female':0})
    
##model=DecisionTreeClassifier().fit(train_data[['Pclass', 'Sex_boolean', 'Age', 'SibSp', 'Parch', 'Fare']],train_data['Survived'])

##model.predict(test_data['Survived'])

We get our second error. Something to do with a Nan. If you see the dataframe using .describe function, you will see a lot of null values in the age column and there is one null in one fare col. We need to replace it with the mean values. Now we write the complete 5-line code and submit it. 

In [ ]:
for df in [train_data, test_data]:
    df['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
    df['Age'].fillna(train_data['Age'].mean(), inplace=True)

##predictions=DecisionTreeClassifier().fit(train_data[['Pclass', 'Sex_boolean', 'Age', 'SibSp', 'Parch', 'Fare']],train_data['Survived']).predict(test_data[['Pclass', 'Sex_boolean', 'Age', 'SibSp', 'Parch', 'Fare']])

##Append the predictions to the PassengerID and convert to CSV
pd.DataFrame({'PassengerId':test_data['PassengerId'],'Survived':predictions}).to_csv('KaggleOutput', index=False)

To impute the missing values, we take the mean and replace it in the dataframe. So df['Fare'].mean replaces the missing fare. Ditto for missing age. Note that we have to set the test data aside and not look at it for any data wrangling else some characteristic of the test data 'leaks' to the training data and test_data is no longer independent. For this reason we dont calculate the **test_data mean** but we USE THE TRAIN_DATA TO CALCULATE the mean and use it to fill missing values in **both** test and train_data. 

Let us submit the file again.

Oh dear 72.7%. It has actually gone low. Shows that our model badly overfitted the training data. Overfitting is somewhat similar to taking an oversmart decision. The model has applied too much brains when probably there was no need for it to. It used the train_data to come up with some really complicated model. This model fits the training data well (overfits it as a matter of fact) but when applied to data outside, it begins to fall apart. Titanic is a simple data set. The model should have kept things simple. We will help it keep things simple by cutting down the number of features. So instread of ['Pclass', 'Sex_boolean', 'Age', 'SibSp', 'Parch', 'Fare'], we will just stick to Pclassand Sex_boolean.

In [ ]:
predictions=DecisionTreeClassifier().fit(train_data[['Pclass', 'Sex_boolean']],train_data['Survived']).predict(test_data[['Pclass', 'Sex_boolean']])

##Append the predictions to the PassengerID and convert to CSV
pd.DataFrame({'PassengerId':test_data['PassengerId'],'Survived':predictions}).to_csv('KaggleOutput', index=False)

Yipee! 75.6%. The honour is preserved. Now let us see 'how' this model has worked? This is one of the rare models where we can actually make out what is happening inside the machine and hence this is very very important.

This is actually 1 line of code.

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

model=DecisionTreeClassifier().fit(train_data[['Pclass','Sex_boolean']],train_data['Survived'])

plt.figure(figsize=(40,20))  
_ = tree.plot_tree(model, feature_names = ['Pclass', 'Sex_boolean'], filled=True, fontsize=30, rounded = True)

- AI without ML - Humans provide the rules to the machines
- AI with ML - Humans provide only the data. The machines learn the rules themselves.

When Arthur Samuel defined ML in 1959 as as the “a field of study that gives computers the ability to learn without being explicitly programmed”, little did he know that a few decades later, it takes just 1 line of code to understand exactly what he meant.

See the decision tree image to understand how the machine made up the rules. Ignore the gini business for now. Let us look at the rest of the data. It is very much similar to the 10 question game we played as kids. For now let us not worry about HOW the model is built. Let us assume that the model building is done and look at what happemns when a new observation comes in...say a 'male' and he belongs to 3rd class. Let us see how the model now decides whether he survives or not survives. 

Start from the top node. There are 891 samples reaching this decision point (basically the entire training set). The 'value' is nothing but the label. 549 expired and 342 survived out of these 891. Now let us see the condition itself - Is Sex_boolen<=0.5? No it is male and sex=1. True is left and False is right so we move to the below level to the node to the right. 

In next node, we have 577 samples which corresponds to the 577 males in the dataset. Out of this 468 expired and 109 survived. The question at this decision point is: Is Pclass<=1.5. Since our man's Pclass is =3 so it moves below to the node on the right side. There it checks for Pclass<=2.5  and then moves to the last level on the right. There are 347 samples here which means in the training data set there are 347 rows which contain Pclass=3 and Sex=Male. Now check the value column. It shows [300, 47]. This means out of 347 people, 300 expired and 47 survive. This is a leaf node. There is nowhere else to go to. So the model has to give a decision now. What decision do you think it will give? Obviously at this node 300 expire and 47 survive out of 347. Naturally the model or any rational being would assume that if somebody reaches this node it is more likely that he will expire. So the answer returned by model is 'Expire'. But you do know that there are 47 people who survive. They will be predicted WRONGLY by the model. There is nothing you can do about it..other than improving the model or trying a new model. In fact this gini business is all about giving an idea of 'by how much the model could be wrong' at that node. It is like an 'error' factor. There is a simple formula for it but we will not bother about that right now

Now do the same for a Pclass1 female. You will get SURVIVED. You end up on the left bottom most node (leaf). The samples there is 94. This means there are 94 females who travelled in Pclass 1. The value field shows that 3 expire and 91 survive. Naturally the node predicts 'Survival'. Look at the gini value here. As we discussed it is a simple measure of error. What do you think it should be - high or low? Obviously the possibility of error here is very very low. So the gini is .062 compared to the previous flow (Pclass3 Male) where gini was .234.

This explains the whole magical genie business. It is nothing but a measure of the error at that node. Another way to look at it is to say that it is a (inverse) measure of confidence of the model at that node. 

Scikit uses a binary decision tree called CART. In other words, every level has only 2 decisions - Yes or No. if a sample satisfies the condition, then it goes to the LEFT branch (if exists), otherwise it goes RIGHT

For a model to be good, the ginis have to be as low as possible. Let us quickly see how the model tries to keep the gini low. When creating the first node, the model tries to analyse all features.  In this case we have only 2 features - Sex and Pclass. The model takes each feature individually, calculates the Gini for every possible split in that feature and then selects the feature/split which leads to lowest Gini score. Between Sex and Pclass which do you think will have a better Gini score? If we split by Sex, and predict that females live and males die, we will not be widely off the mark right? The gini would be decent here. Now instead of Sex, let us consider Pclass. If we blindly say Pclass 1 survives and Pclass2,3 expires, we may not get a very high gini score. This is because there are fair number of females in Pclass 2,3 who survive. Even if we predict that Pclass 1,2 survive and all Pclass 3 expire, the gini is still lower than the earlier case (the one with Sex feature where we assume all males die and females live). So the model chooses the feature and the split that MAKES MOST SENSE and in this case it is Sex - m or f. This is the first level. 

Now the model moves on to the second level. Obviously there is only one feature left - Pclass. But what should be the binary split that will yield the best gini? Interestingly the answer is different in case of males and females. In case of females that split is Pclass3 versus Pclass 1,2. In case of males it is Pclass 1 versus Pclass2,3. This is a clever model because Pclass 2 females almost always survive (like Pclass 1 females) and Pclass 2 males have expiry rates similar to Pclass 3 males.

The way the tree is contructed is not too different from the way we construct our questions when we played the 10 question game as kids. The first question should be the one from which we glean the max information possible, then the next question and so on..

Above is a simple intuitive explanation of a decision tree and gives a feel of how an ML model generates code from data. You may not have the luxury of seeing this type of inner working for every model. But hopefully this gives you an idea of how any model works. There is an error function and the model works to minimize this error function. Another thing both man and machine try to minimize (esp in these times) is Cost. So this error minimization is more popularly called Cost minimization or 'Cost function' and there are several interesting ways to do it.

In [ ]:
##Scikit allows you to print the rules to see what is happening
from sklearn.tree.export import export_text

tree_rules = export_text(model, feature_names=['Pclass', 'Sex_boolean'])
print(tree_rules)

Note that if you have an older version of Scikit, the above graphs/trees may not be generated and you can try graphviz as below to generate the decision tree

- from sklearn.tree import export_graphviz
- from subprocess import check_call
- from IPython.display import Image
- from sklearn import tree

tree.export_graphviz(model,out_file='TitanicDT.dot', feature_names = ['Pclass', 'Sex_boolean'], class_names = ['Expired', 'Survived'], rounded = True, filled= True )

This needs the graphviz on your local machine. Do a brew install graphviz. After that below commands should work.
- check_call(['dot','-Tpng','tree1.dot','-o','TitanicDT.png'])
- Image(filename = 'TitanicDT.png')

Up Next:
- Advanced data exploration and visualization - https://www.kaggle.com/allohvk/captivating-conversations-with-the-titanic-dataset
- Advanced Strategy to fill missing ages - https://www.kaggle.com/allohvk/titanic-missing-age-imputation-tutorial-advanced
- Advanced feature engineering
- Final ensemble